In [1]:
import numpy as np
import pandas as pd
import math
import os

In [2]:
def create_ans(read_file_path,read_record_path):
    dir_name = read_file_path + "/ans"
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    record_fault = np.genfromtxt((read_record_path+"/start_fault_list.csv"), delimiter=',', dtype=np.int32)
    record_file  =  np.genfromtxt((read_record_path+"/start_plant_list.csv"), delimiter=',', dtype=np.int32)
    record_bound = np.genfromtxt((read_record_path+"/record_bound.csv"), delimiter=',', dtype=np.int32)
    
    record_vali = np.genfromtxt((read_file_path+"/pred_start/record_vali.csv"), delimiter=',', dtype=np.int32)
    record_start_cutpoint = np.genfromtxt((read_file_path+"/pred_start/record_cutpoint.csv"), delimiter=',', dtype=np.float32)
    #record_end_cutpoint = np.genfromtxt((read_file_path+"/pred_end/record_cutpoint.csv"), delimiter=',', dtype=np.float32)
    
    for n_index in range(0,2):
        print( "index: %s   total: %s"%(n_index, len(record_vali)))
        if record_vali[n_index] == 1:
            st_threshold = record_start_cutpoint[n_index]
            print(st_threshold)
            
            bound = record_bound[n_index]
            pred_start_name = read_file_path + "/pred_start/start_pred_5conv" + str(record_file[n_index]) + "_fault" + str(record_fault[n_index]) + ".csv"
            pred_end_name = read_file_path + "/pred_end/end_pred_5conv" + str(record_file[n_index]) + "_fault" + str(record_fault[n_index]) + ".csv"
            pred_start = pd.read_csv(pred_start_name)
            pred_end = pd.read_csv(pred_end_name)
            
            ## find start time
            st_set = []
            for index in range( 0,math.floor((pred_start.time[pred_start.shape[0]-1]-pred_start.time[0])/6)-1):
                max_prob = 0
                index_start = 0
                section = range( int(pred_start.time[0])+(index)*6, int(pred_start.time[0])+(index+1)*6)
                sec_start = pred_start.loc[pred_start.time.isin(section),:]
                if sec_start.shape[0] > 0:
                    sec_start.index = range(sec_start.shape[0])
                    for sec_index in sec_start.index:
                        if sec_start.one[sec_index] > max_prob:
                            max_prob = sec_start.one[sec_index]
                            index_start = sec_index

                    if(max_prob > st_threshold):
                        st_set.append(int(sec_start.time[index_start]) )

            ##find the end time 
            ans_set = pd.DataFrame({'start_time' : [], 'end_time': []})
            for st_time in st_set:
                max_prob = 0
                index_end = 0
                section = range(st_time+1, st_time+bound+1)
                sec_end = pred_end.loc[pred_end.time.isin(section),:]
                if sec_end.shape[0] > 0:
                    sec_end.index = range(sec_end.shape[0])
                    for sec_index in sec_end.index:
                        if sec_end.one[sec_index] > max_prob:
                            max_prob = sec_end.one[sec_index]
                            index_end = sec_index
                    ans_set.loc[ans_set.shape[0]] = [st_time,int(sec_end.time[index_end])]
            
            print(ans_set.shape)
            ans_set_name = read_file_path + "/ans/ans_plant"+ str(record_file[n_index]) + "_fault" + str(record_fault[n_index])+".csv"
            ans_set.to_csv(ans_set_name, index = False, sep = ",")
            

In [3]:
read_file_path ="/media/joeytu/82F6C88FF6C8853F/PHM2017/20170831DEMO/file"
read_record_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_nosample/file"

create_ans(read_file_path,read_record_path)

index: 0   total: 2
0.819826
(2061, 2)
index: 1   total: 2
0.894259
(40, 2)
